In [3]:
# langchain + RAG
import pandas as pd
from langchain_chroma import Chroma

from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

emdeding_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
tokenizer_opt = {
    "max_length": 512, 
    "truncation": True,
}

df = pd.read_csv("C://Users//Leo//code//ch09_랭체인_기초//test02//data/data.csv", encoding="utf-8")
texts = df["text"].tolist()
docs = [Document(page_content=text) for text in texts]
docs

[Document(metadata={}, page_content='Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다.'),
 Document(metadata={}, page_content='PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다.'),
 Document(metadata={}, page_content='웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다.'),
 Document(metadata={}, page_content='불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다.'),
 Document(metadata={}, page_content='클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다.'),
 Document(metadata={}, page_content='포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다.'),
 Document(metadata={}, page_content='검사 장비는 매일 초기화 후 기능 점검을 실시합니다.'),
 Document(metadata={}, page_content='주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다.'),
 Document(metadata={}, page_content='X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다.'),
 Document(metadata={}, page_content='로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다.'),
 Document(metadata={}, page_content='공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.'),
 Document(metadata={}, page_content='동일 Lot 내에서 불량이 5개 이상 발생하면 전수 검사 실시합니다,'),
 Document(metadata={}, page_content='물류 이송 로봇은 경로 장애 감지 시 자동 우회합니다.'),
 Document(metadata={}, page_content='이

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name = emdeding_id)
save_vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chromaDB1"
)
load_vectordb = Chroma(
    persist_directory="./chromaDB1",
    embedding_function=embedding_model
)
question = "클린룸 입장시 주의 사항 알려줄래?"
retriever = load_vectordb.as_retriever(search_kwargs={"k":3})
relevant_docs = retriever.invoke(question)
best_context = relevant_docs[0].page_content
print("References :", best_context)

References : 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.


In [5]:
relevant_docs

[Document(id='f6f7f7d6-28c1-4b64-a09d-0dd5925f4f68', metadata={}, page_content='공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.'),
 Document(id='e8d38da8-a23d-4ee5-afa4-0f4c4aa5a40e', metadata={}, page_content='공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.'),
 Document(id='b1184416-11d9-4925-ab2d-9eb2c6553b6e', metadata={}, page_content='공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.')]

In [ ]:
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_id)
text_gen = pipeline(
    task="question-answering",
    model=qa_model,
    tokenizer=qa_tokenizer,
    max_length=512,
    truncation=True,
    do_sample=True,
    device = -1
)

result = text_gen(question=question, context=best_context)
print("Answer :", result['answer'])

Device set to use cpu


Answer : 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다


In [ ]:
# 앞에것과 동일하지만, transformer 미사용

import pandas as pd
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_chroma import Chroma
from langchain_core.documents import Document

from langchain_huggingface import HuggingFacePipeline

emdeding_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
tokenizer_opt = {
    "max_length": 512, 
    "truncation": True,
}

df = pd.read_csv("C://Users//Leo//code//ch09_랭체인_기초//test02//data/data.csv", encoding="utf-8")
texts = df["text"].tolist()
docs = [Document(page_content=text) for text in texts]
embedding_model = HuggingFaceEmbeddings(model_name=emdeding_id)
save_vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chromaDB2"
)
load_vectordb = Chroma(
    persist_directory="./chromaDB2",
    embedding_function=embedding_model
)

qa_pipeline = pipeline(
    task="question-answering",
    model=qa_model_id,
    tokenizer_kwargs=tokenizer_opt,
    device = -1
)

question = "클린룸 입장 시 주의사항 알려줄래?"
retriever = load_vectordb.as_retriever(search_kwargs={"k":1})
relevant_docs = retriever.invoke(question)
best_context = relevant_docs[0].page_content
print("참고 문서:", best_context)

result = qa_pipeline(
    question=question,
    context=best_context
)
print("Answer:", result['answer'])

Device set to use cpu


Context: 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.
Answer: 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다


In [2]:
# GEN 추가

import pandas as pd
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline

embedding_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer_opt = {
    "max_length": 512,  
    "truncation": True,
}
gen_model_id = "EleutherAI/polyglot-ko-1.3b"

embedding_model = HuggingFaceEmbeddings(model_name=embedding_id)
load_vectordb = Chroma(
    persist_directory="./chromaDB2",
    embedding_function=embedding_model
)   

question = "클린룸 입장 시 주의사항 알려줄래?"
retriever = load_vectordb.as_retriever(search_kwargs={"k":3})
relevant_docs = retriever.invoke(question)
best_context = relevant_docs[0].page_content
print("References :", best_context)


References : 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.


In [3]:
# KoElectra QA 모델 로드
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_id)

text_gen = pipeline(
    task="question-answering",
    model=qa_model,
    tokenizer=qa_tokenizer,
    tokenizer_kwargs=qa_tokenizer_opt,
    max_length=512,
    truncation=True,
    do_sample=False,
    device=-1
)

result = text_gen(question=question, context=best_context)
print("Answer :", result['answer'])

Device set to use cpu


Answer : 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다


In [6]:
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_id)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_id)
gen_pipeline = pipeline(
    task="text-generation",
    model=gen_model,
    tokenizer=gen_tokenizer,
    max_length=200,
    truncation=True,
    do_sample=True,
    temperature=0.1,
    device=-1
)
gen_llm = HuggingFacePipeline(pipeline=gen_pipeline)

prompt = PromptTemplate.from_template("""
너는 똑똑한 제조 전문가야. 아래의 질문과 정답을 바탕으로 자연스럽고 친절하게 응답해줘.
    Question: {question}
    Answer: {answer}
    자연어 응답:
""")
chain = prompt|gen_llm

final_response = chain.invoke({
    "question": question,
    "answer": result['answer']
})
print("자연어 응답:")
print(final_response)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


자연어 응답:

너는 똑똑한 제조 전문가야. 아래의 질문과 정답을 바탕으로 자연스럽고 친절하게 응답해줘.
    Question: 클린룸 입장 시 주의사항 알려줄래?
    Answer: 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다
    자연어 응답:
    Question: 클린룸에서 공정 이탈 시 어떻게 해야 하는지 알려줄래?    Answer: 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.    Question: 클린룸에서 공정 이탈 시 어떻게 해야 하는지 알려줄래?    Answer: 공정 이탈 발생 시 Shift 리더에게 즉시 보고합니다.    Question: 클린룸에서 공정 이탈 시 어떻게 해야 하는지 알려줄래?    Answer: 공정 이탈 발생 시


In [18]:
# RAG + GEN + History
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from uuid import uuid4

model_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text_gen = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    truncation=False,
    do_sample=True,
    return_full_text=False,
    temperature=0.7,
)
llm = HuggingFacePipeline(pipeline=text_gen)

prompt = PromptTemplate.from_template(
    "지금까지의 이야기:\n{history}\n\n이제 '{animal}'에 대한 이야기를 써 주세요:\n"
)
chat_histories = {}
def get_history(session_id):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]


Device set to use cpu


In [19]:
def run():
    sesseion_id = str(uuid4())
    print("동물 이름을 입력해 이야기를 시작하세요.")

    while True:
        animal = input("\n 동물 이름 또는 질문 입력(종료하려면 '종료'): ").strip()
        if animal in ["종료", "exit", "quit"]:
            print("이야기를 종료합니다.")
            break

        history = get_history(sesseion_id)

        if history.messages:
            history_text = "\n".join(
                f"{msg.type.upper()}: {msg.content}" for msg in history.messages)
            prompt_input = prompt.format(history=history_text, animal=animal)
        else:
            prompt_input = f"'아주 먼 옛날 {animal} 한 마리가 살고 있었습니다.그러던 어느날'"

        final_response = llm.invoke(prompt_input)
        print(f"\n생성된 이야기:\n{final_response}")

        history.add_user_message(animal)
        history.add_ai_message(final_response)
        

In [20]:
run()

동물 이름을 입력해 이야기를 시작하세요.

생성된 이야기:
 그 토끼를 본 토끼의 어머니는 깜짝 놀라 그 토끼의 머리를 물어뜯어 물었습니다." 토끼는 그 토끼의 목을 물어뜯어 죽인 후에 그 토끼의 머리에다 물을 뿌렸습니다.
이때 그 토끼는 그 토끼의 머리 위로 올라가서 그 토끼의 머리 위에 올라가서 그 토끼의 머리를 물어뜯었습니다.
이처럼 토끼의 머리 위에 올라서 물리는 물고기의 습성은, 그 물고기의 습성이 물릴 때 그 물고기의 습성을 씻어 주는 습성을 담고

생성된 이야기:
"우리는 '사자'에 대한 이야기를 잘 쓰기 위해서는, 그 물고기의 습성을 풀어 내야 합니다."
1. '사자'에 대하여 이야기하기 위해서는, 그 물고기의 습성을 풀어 주어야 합니다.
그 물고기의 습성은, 그 물고기의 습성이 어떤 것인지 잘 알기 위해서는, 그 물고기의 습성이 어떤 것인지 잘 알고 있어야 합니다.
2. 물고기에게 말을 걸어 볼 때에는, 그 물고기의 습성이 어떤 것인지 잘 설명하고 있어야 합니다.
3. 물고기의 습성이 어떻게 되는지를 알아내려면, 그 물고기의

생성된 이야기:
"고양이에 대해서 이야기해 주세요."
"고양이에게 말을 걸어 볼 때에는, 그 물고기의 습성이 어떤 것인지 잘 이해해야 합니다."
"고양이에게 말을 걸어 볼 때에는, 그 물고기의 습성이 어떤 것인지 잘 알고 있어야 합니다."
3. 물고기에게 말을 걸어 볼 때에는, 그 물고기의
이 물음에 대답해 주어야 합니다.
"고양이에 대해서 이야기해 주세요."
4. 물고기의
이 물음에 대답해 주십시오.
"고양이에게 말을 걸어 볼
이야기를 종료합니다.
